In [4]:
import requests
import json
import os

# URL do JSON do openbible (versão em inglês – KJV)
url = "https://raw.githubusercontent.com/churchstudio-org/openbible/refs/heads/main/en_US/bible.json"

# 1. Fazer o download
resp = requests.get(url)
resp.raise_for_status()
data = resp.json()

# 2. (Opcional) checar estrutura
# Por exemplo: contar livros, capítulos, versículos
print("Número de livros:", len(data))
# Exibe quantos capítulos no primeiro livro
print("Capítulos no livro 1:", len(data[0]))
# Exibe quantos versículos no capítulo 1 do livro 1
print("Versículos no livro1, capítulo1:", len(data[0][0]))
# Exemplo de versículo:
print("Primeiro versículo:", data[0][0][0])

# 3. (Opcional) filtrar ou modificar se necessário
# Por exemplo, você poderia querer excluir livros apócrifos, ou remover notas etc.
# Aqui vamos assumir que o arquivo já está correto para seus propósitos.

# 4. Salvar como kjv.json
output_path = "kjv.json"
with open(output_path, "w", encoding="utf-8") as f:
    json.dump(data, f, ensure_ascii=False, indent=2)

print(f"Salvo como {output_path}, pronto para usar no pipeline de tradução.")


Número de livros: 66
Capítulos no livro 1: 50
Versículos no livro1, capítulo1: 31
Primeiro versículo: In the beginning God created the heaven and the earth.
Salvo como kjv.json, pronto para usar no pipeline de tradução.


In [ ]:
!pip install transformers torch sentencepiece tqdm

In [6]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

import json
from transformers import MarianMTModel, MarianTokenizer
from tqdm import tqdm

# ---------------------------------------------------------
# 1. Escolher modelo (aqui MarianMT en->pt, mas pode trocar)
# ---------------------------------------------------------
model_name = "Helsinki-NLP/opus-mt-tc-big-en-pt"
tokenizer = MarianTokenizer.from_pretrained(model_name)
model = MarianMTModel.from_pretrained(model_name).to(device)

# ---------------------------------------------------------
# 2. Carregar texto bíblico em inglês (KJV)
#    -> Estrutura tridimensional [livro][capítulo][versículo]
# ---------------------------------------------------------
with open("kjv.json", "r") as f:
    kjv = json.load(f)

# Exemplo da estrutura (Genesis 1:1-2)
print(kjv[0][0][0:2])

# ---------------------------------------------------------
# 3. Função para traduzir uma lista de versículos
# ---------------------------------------------------------
def translate_batch(verses, batch_size=8):
    translations = []
    for i in range(0, len(verses), batch_size):
        batch = verses[i:i+batch_size]
        inputs = tokenizer(batch, return_tensors="pt", padding=True, truncation=True).to(device)
        translated = model.generate(**inputs)
        out = [tokenizer.decode(t, skip_special_tokens=True) for t in translated]
        translations.extend(out)
    return translations

# ---------------------------------------------------------
# 4. Traduzir toda a Bíblia
# ---------------------------------------------------------
bible_pt = []

for book in tqdm(kjv, desc="Livros"):
    book_pt = []
    for chapter in book:
        chapter_pt = translate_batch(chapter)
        book_pt.append(chapter_pt)
    bible_pt.append(book_pt)

# ---------------------------------------------------------
# 5. Salvar no formato openbible
# ---------------------------------------------------------
with open("kjv_pt.json", "w", encoding="utf-8") as f:
    json.dump(bible_pt, f, ensure_ascii=False, indent=2)

print("Tradução final salva em kjv_pt.json ✅")


cuda
['In the beginning God created the heaven and the earth.', 'And the earth was without form, and void; and darkness was upon the face of the deep. And the Spirit of God moved upon the face of the waters.']


Livros: 100%|██████████| 66/66 [38:29<00:00, 35.00s/it]

Tradução final salva em kjv_pt.json ✅


In [7]:
from google.colab import files
import shutil

files.download('kjv_pt.json')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [8]:
import json
import os
from enum import Enum

# --- Enum já definido acima ---
class BookOf(Enum):
    Genesis = 1
    Exodus = 2
    Leviticus = 3
    Numbers = 4
    Deuteronomy = 5
    Joshua = 6
    Judges = 7
    Ruth = 8
    Samuel1 = 9
    Samuel2 = 10
    Kings1 = 11
    Kings2 = 12
    Chronicles1 = 13
    Chronicles2 = 14
    Ezra = 15
    Nehemiah = 16
    Esther = 17
    Job = 18
    Psalms = 19
    Proverbs = 20
    Ecclesiastes = 21
    SongOfSolomon = 22
    Isaiah = 23
    Jeremiah = 24
    Lamentations = 25
    Ezekiel = 26
    Daniel = 27
    Hosea = 28
    Joel = 29
    Amos = 30
    Obadiah = 31
    Jonah = 32
    Micah = 33
    Nahum = 34
    Habakkuk = 35
    Zephaniah = 36
    Haggai = 37
    Zechariah = 38
    Malachi = 39
    Matthew = 40
    Mark = 41
    Luke = 42
    John = 43
    Acts = 44
    Romans = 45
    Corinthians1 = 46
    Corinthians2 = 47
    Galatians = 48
    Ephesians = 49
    Philippians = 50
    Colossians = 51
    Thessalonians1 = 52
    Thessalonians2 = 53
    Timothy1 = 54
    Timothy2 = 55
    Titus = 56
    Philemon = 57
    Hebrews = 58
    James = 59
    Peter1 = 60
    Peter2 = 61
    John1 = 62
    John2 = 63
    John3 = 64
    Jude = 65
    Revelation = 66

# --- Configuração ---
input_file = "kjv_pt.json"
output_dir = os.path.splitext(input_file)[0]  # "kjv_pt"

# Criar diretório de saída se não existir
os.makedirs(output_dir, exist_ok=True)

# --- Carregar JSON ---
with open(input_file, "r", encoding="utf-8") as f:
    bible = json.load(f)

# --- Salvar cada livro ---
for idx, book_data in enumerate(bible, start=1):
    try:
        book_name = BookOf(idx).name
    except ValueError:
        book_name = f"Book{idx}"  # fallback se índice não existir no enum

    output_path = os.path.join(output_dir, f"{book_name.lower()}.json")

    with open(output_path, "w", encoding="utf-8") as f:
        json.dump(book_data, f, ensure_ascii=False, indent=2)

    print(f"✔ Livro salvo: {output_path}")

✔ Livro salvo: kjv_pt/genesis.json
✔ Livro salvo: kjv_pt/exodus.json
✔ Livro salvo: kjv_pt/leviticus.json
✔ Livro salvo: kjv_pt/numbers.json
✔ Livro salvo: kjv_pt/deuteronomy.json
✔ Livro salvo: kjv_pt/joshua.json
✔ Livro salvo: kjv_pt/judges.json
✔ Livro salvo: kjv_pt/ruth.json
✔ Livro salvo: kjv_pt/samuel1.json
✔ Livro salvo: kjv_pt/samuel2.json
✔ Livro salvo: kjv_pt/kings1.json
✔ Livro salvo: kjv_pt/kings2.json
✔ Livro salvo: kjv_pt/chronicles1.json
✔ Livro salvo: kjv_pt/chronicles2.json
✔ Livro salvo: kjv_pt/ezra.json
✔ Livro salvo: kjv_pt/nehemiah.json
✔ Livro salvo: kjv_pt/esther.json
✔ Livro salvo: kjv_pt/job.json
✔ Livro salvo: kjv_pt/psalms.json
✔ Livro salvo: kjv_pt/proverbs.json
✔ Livro salvo: kjv_pt/ecclesiastes.json
✔ Livro salvo: kjv_pt/songofsolomon.json
✔ Livro salvo: kjv_pt/isaiah.json
✔ Livro salvo: kjv_pt/jeremiah.json
✔ Livro salvo: kjv_pt/lamentations.json
✔ Livro salvo: kjv_pt/ezekiel.json
✔ Livro salvo: kjv_pt/daniel.json
✔ Livro salvo: kjv_pt/hosea.json
✔ Livro 

In [9]:
from google.colab import files
import shutil

shutil.make_archive('kjv_pt', 'zip', 'kjv_pt')
files.download('kjv_pt.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>